#### pytorch搭建神经网络模型，实现对KMNIST数据集的训练

In [41]:
#导入包
import torch
from torchvision.datasets import KMNIST
from torchvision.transforms import ToTensor 
from torch.utils.data import DataLoader
import torch.nn as nn



In [ ]:
# 定义超参数
LR = 1e-3
epochs = 100#训练轮次，100轮左右损失率降到了1以下
BATCH_SIZE = 128  #每批64张

In [54]:
#数据集加载
train_data = KMNIST(root='./kmnist_data', train=True, download=True, transform=ToTensor())
test_data = KMNIST(root='./kmnist_data', train=False, download=True, transform=ToTensor())
print(train_data[0])

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.4627, 1.0000, 1.0000, 0.4863, 0.0039,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.1412, 0.9333, 1.0000, 0.5725, 0.0078, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0471, 0.7961, 1.0000, 0.8627, 0.0549, 0.0000, 0.0000,
          0.0000, 0.0431, 0.5176, 0.3725, 0.7333, 0.3725, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.5843, 1.0000, 0.9843, 0.2588, 0.0000, 0.0000, 0.0000,
          0.0275, 0.6588, 0.5333, 0.0392, 0.8745, 0.9608

In [55]:
#数据分批次处理
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True) # shuffle：随机打乱数据


In [56]:
# 模型创建
model = nn.Sequential(
    nn.Linear(784, 128),  #输入层
    nn.Sigmoid(),     #隐藏层
    nn.Linear(128, 10)  #输出层
)
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
#损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数

#优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR) #SGD优化器

###加上训练轮次
# 训练加速，数据加载器，分批次
for epoch in range(epochs):
    for data, target in train_dl:
        #前向运算
        output = model(data.reshape(-1, 784))
        #计算损失
        loss = loss_fn(output, target) #计算梯度
        #反向传播
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()  #计算梯度（参数.grad）
        optimizer.step() #更新参数
    print(f'Epoch:{epoch}, Loss:{loss.item()}')

Epoch:0, Loss:2.292895555496216
Epoch:1, Loss:2.2866485118865967
Epoch:2, Loss:2.2662768363952637
Epoch:3, Loss:2.2562026977539062
Epoch:4, Loss:2.236692190170288
Epoch:5, Loss:2.224648952484131
Epoch:6, Loss:2.2222096920013428
Epoch:7, Loss:2.190190315246582
Epoch:8, Loss:2.1709864139556885
Epoch:9, Loss:2.1737606525421143
Epoch:10, Loss:2.138089418411255
Epoch:11, Loss:2.1235227584838867
Epoch:12, Loss:2.097942590713501
Epoch:13, Loss:2.060696840286255
Epoch:14, Loss:2.0638186931610107
Epoch:15, Loss:2.055426836013794
Epoch:16, Loss:1.9798721075057983
Epoch:17, Loss:1.9939236640930176
Epoch:18, Loss:1.9765390157699585
Epoch:19, Loss:1.9552359580993652
Epoch:20, Loss:1.896746277809143
Epoch:21, Loss:1.8252272605895996
Epoch:22, Loss:1.9004902839660645
Epoch:23, Loss:1.8050726652145386
Epoch:24, Loss:1.788552165031433
Epoch:25, Loss:1.739066481590271
Epoch:26, Loss:1.752973198890686
Epoch:27, Loss:1.8368171453475952
Epoch:28, Loss:1.7913637161254883
Epoch:29, Loss:1.6404606103897095
Ep

In [59]:
# 测试
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

correct = 0
total = 0

with torch.no_grad(): #不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 784))
        _, predicted = torch.max(output, 1) #返回第一个维度的最大值张量
        total += target.size(0) #返回张量的样本量
        correct += (predicted == target).sum().item()
print(f'Accuracy: {correct/total*100}%')

Accuracy: 76.03%
